### Data Preparation for Bayesian Filter

install dependencies....

In [2]:
!pip install beautifulsoup4

You are using pip version 9.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
!pip install wordcloud

You are using pip version 9.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from math import log, sqrt
import pandas as pd
import numpy as np
import os
import io
%matplotlib inline

### This function reads the files in the specified path.

In [8]:
def read_files(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root,filename)
            
            inBody = False
            lines = []
            
            f = io.open(path, 'r', encoding = 'latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message
                    

### This function is for creating dataframes from the dataset in question... it also classifies the emails into either ham or spam

In [9]:
def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in read_files(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)
    return pd.DataFrame(rows, index=index)

aggregate all the emails into a single dataframe. The key is: ham = 0, spam = 1

In [30]:
data = pd.DataFrame({'message':[],'class':[]})
data = data.append(dataFrameFromDirectory("ham-and-spam-dataset/ham",0))
data = data.append(dataFrameFromDirectory("ham-and-spam-dataset/spam",1))
data

,class,message
ham-and-spam-dataset/ham\0001.ea7e79d3153e7469e7a9c3e0af6a357e,0.0,"Date: Wed, 21 Aug 2002 10:54:46 -05..."
ham-and-spam-dataset/ham\0002.b3120c4bcbf3101e661161ee7efcb8bf,0.0,"Martin A posted:\n\nTassos Papadopoulos, the G..."
ham-and-spam-dataset/ham\0003.acfc5ad94bbd27118a0d8685d18c89dd,0.0,Man Threatens Explosion In Moscow \n\n\n\nThur...
ham-and-spam-dataset/ham\0004.e8d5727378ddde5c3be181df593f1712,0.0,Klez: The Virus That Won't Die\n\n \n\nAlready...
ham-and-spam-dataset/ham\0005.8c3b9e9c0f3f183ddaf7592a11b99957,0.0,"On Wed Aug 21 2002 at 15:46, Ulises Ponce wrot..."
ham-and-spam-dataset/ham\0006.ee8b0dba12856155222be180ba122058,0.0,"> in adding cream to spaghetti carbonara, whi..."
ham-and-spam-dataset/ham\0007.c75188382f64b090022fa3b095b020b0,0.0,\n\n> I just had to jump in here as Carbonara ...
ham-and-spam-dataset/ham\0008.20bc0b4ba2d99aae1c7098069f611a9b,0.0,The Scotsman - 22 August 2002\n\n\n\n Playboy ...
ham-and-spam-dataset/ham\0009.435ae292d75abb1ca492dcc2d5cf1570,0.0,Martin Adamson wrote:\n\n> \n\n> Isn't it just...
ham-and-spam-dataset/ham\0010.4996141de3f21e858c22f88231a9f463,0.0,The Scotsman\n\n\n\n Thu 22 Aug 2002 \n\n\n\n ...


The below cell shows the number of rows in our dataframe.

In [31]:
total_mails = data['message'].shape[0]
total_mails

3052

The function below extracts all text from html text present in all emails.

In [54]:
from bs4 import BeautifulSoup

def func(df):
    soup = BeautifulSoup(df['message'], "html.parser").find()
    if bool(soup):
#         is_html = soup.find('html')
#         df['html'] = 'True'
        
#         if bool(is_html):
#             df['mezz'] = 'True'
#         soup = BeautifulSoup(df['message'], "html.parser")
    
        text = soup.find_all(text=True)
        print(text)
        df['message'] = text
        
        return text
    else:
        
        return df['message'] 

In [40]:
new_data = data.reset_index()
new_data

,index,class,message
0,ham-and-spam-dataset/ham\0001.ea7e79d3153e7469...,0.0,"Date: Wed, 21 Aug 2002 10:54:46 -05..."
1,ham-and-spam-dataset/ham\0002.b3120c4bcbf3101e...,0.0,"Martin A posted:\n\nTassos Papadopoulos, the G..."
2,ham-and-spam-dataset/ham\0003.acfc5ad94bbd2711...,0.0,Man Threatens Explosion In Moscow \n\n\n\nThur...
3,ham-and-spam-dataset/ham\0004.e8d5727378ddde5c...,0.0,Klez: The Virus That Won't Die\n\n \n\nAlready...
4,ham-and-spam-dataset/ham\0005.8c3b9e9c0f3f183d...,0.0,"On Wed Aug 21 2002 at 15:46, Ulises Ponce wrot..."
5,ham-and-spam-dataset/ham\0006.ee8b0dba12856155...,0.0,"> in adding cream to spaghetti carbonara, whi..."
6,ham-and-spam-dataset/ham\0007.c75188382f64b090...,0.0,\n\n> I just had to jump in here as Carbonara ...
7,ham-and-spam-dataset/ham\0008.20bc0b4ba2d99aae...,0.0,The Scotsman - 22 August 2002\n\n\n\n Playboy ...
8,ham-and-spam-dataset/ham\0009.435ae292d75abb1c...,0.0,Martin Adamson wrote:\n\n> \n\n> Isn't it just...
9,ham-and-spam-dataset/ham\0010.4996141de3f21e85...,0.0,The Scotsman\n\n\n\n Thu 22 Aug 2002 \n\n\n\n ...


split the data into the pareto principle for unbiased model testing.
so 80% will be used for training and 20% for testing.
this will be accomplished by using a random number generator to randomize the order by implementing a uniform distribution randomizer. this removes all chances of having a biased model on account of sequential data.

In [55]:
train_index, test_index = list(), list()
for i in range(data['message'].shape[0]):
    if np.random.uniform(0,1) < 0.80:
        train_index.append(i)
    else:
        test_index.append(i)
        
train_data = new_data.loc[train_index]
test_data = new_data.loc[test_index]

implement the data cleaning of the html based messages
    

In [56]:
train_data['message'] = train_data.apply(func, axis=1)
train_data

['\n\n    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>\n\n\n\n\n\n  | I can\'t reproduce this error.\n\n\n\nFor me it is very repeatable... (like every time, without fail).\n\n\n\nThis is the debug log of the pick happening ...\n\n\n\n18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}\n\n18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury\n\n18:19:04 Ftoc_PickMsgs {{1 hit}}\n\n18:19:04 Marking 1 hits\n\n18:19:04 tkerror: syntax error in expression "int ...\n\n\n\nNote, if I run the pick command by hand ...\n\n\n\ndelta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury\n\n1 hit\n\n\n\nThat\'s where the "1 hit" comes from (obviously).  The version of nmh I\'m\n\nusing is ...\n\n\n\ndelta$ pick -version\n\npick -- nmh-1.0.4 [compiled on fuchsia.cs.mu.OZ.AU at Sun Mar 17 14:55:56 ICT 2002]\n\n\n\nAnd the relevant part o

C:\Users\ikangethe.MO-DE\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.post-gazette.com/columnists/20020905brian5.asp






" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup



[' writes:\n\n>     S> A) Which religion is it that can claim no foul actions in its\n\n>     S> past?  Certainly not Christianity, Islam, etc.\n\n> Rastafari.\n\n> That is, if you concede that of the two founding branches, only the\n\n> one founded by the Nyabingi were legit and the others were\n\n> thinly-veneered anti-colonial hooligans.  \n\n> There is also Vietnamese Buddhism, unless you count setting fire to\n\n> oneself as a "foul action".\n\n\n\nWhat about Tibetan Buddhism BTW?  They seem like an awfully nice bunch\n\nof chaps (and chapesses).\n\n\n\n> When we launched the Native Net in 1989, one of the first things we\n\n> noticed on networking aboriginal groups around the world is that the\n\n> British Army, with the US Army as a proxy by extension, were the\n\n> common thread.  Where neither was present (physically or through\n\n> influence) there /tended/ to be less violence.\n\n> \n\n> The issue in Ireland is complex, but rest assured that religious\n\n> aspects are only 

,index,class,message
0,ham-and-spam-dataset/ham\0001.ea7e79d3153e7469...,0.0,[ Message-ID: <1029945287.4797.TMDA@deep...
1,ham-and-spam-dataset/ham\0002.b3120c4bcbf3101e...,0.0,"Martin A posted:\n\nTassos Papadopoulos, the G..."
2,ham-and-spam-dataset/ham\0003.acfc5ad94bbd2711...,0.0,Man Threatens Explosion In Moscow \n\n\n\nThur...
4,ham-and-spam-dataset/ham\0005.8c3b9e9c0f3f183d...,0.0,"On Wed Aug 21 2002 at 15:46, Ulises Ponce wrot..."
5,ham-and-spam-dataset/ham\0006.ee8b0dba12856155...,0.0,"> in adding cream to spaghetti carbonara, whi..."
7,ham-and-spam-dataset/ham\0008.20bc0b4ba2d99aae...,0.0,The Scotsman - 22 August 2002\n\n\n\n Playboy ...
8,ham-and-spam-dataset/ham\0009.435ae292d75abb1c...,0.0,[ That's one of the worst non-smiles ever. ...
9,ham-and-spam-dataset/ham\0010.4996141de3f21e85...,0.0,The Scotsman\n\n\n\n Thu 22 Aug 2002 \n\n\n\n ...
10,ham-and-spam-dataset/ham\0011.07b11073b53634cf...,0.0,"[ > Date: Wed, 21 Aug 2002 09:51:31 -0400 >..."
11,ham-and-spam-dataset/ham\0012.d354b2d2f24d1036...,0.0,I have been trying to research via SA mirrors ...


In [57]:
train_data

,index,class,message
0,ham-and-spam-dataset/ham\0001.ea7e79d3153e7469...,0.0,[ Message-ID: <1029945287.4797.TMDA@deep...
1,ham-and-spam-dataset/ham\0002.b3120c4bcbf3101e...,0.0,"Martin A posted:\n\nTassos Papadopoulos, the G..."
2,ham-and-spam-dataset/ham\0003.acfc5ad94bbd2711...,0.0,Man Threatens Explosion In Moscow \n\n\n\nThur...
4,ham-and-spam-dataset/ham\0005.8c3b9e9c0f3f183d...,0.0,"On Wed Aug 21 2002 at 15:46, Ulises Ponce wrot..."
5,ham-and-spam-dataset/ham\0006.ee8b0dba12856155...,0.0,"> in adding cream to spaghetti carbonara, whi..."
7,ham-and-spam-dataset/ham\0008.20bc0b4ba2d99aae...,0.0,The Scotsman - 22 August 2002\n\n\n\n Playboy ...
8,ham-and-spam-dataset/ham\0009.435ae292d75abb1c...,0.0,[ That's one of the worst non-smiles ever. ...
9,ham-and-spam-dataset/ham\0010.4996141de3f21e85...,0.0,The Scotsman\n\n\n\n Thu 22 Aug 2002 \n\n\n\n ...
10,ham-and-spam-dataset/ham\0011.07b11073b53634cf...,0.0,"[ > Date: Wed, 21 Aug 2002 09:51:31 -0400 >..."
11,ham-and-spam-dataset/ham\0012.d354b2d2f24d1036...,0.0,I have been trying to research via SA mirrors ...


In [58]:
spam_words = ' '.join(list(train_data[train_data['class'] == 0]['message']))
spamwc = WordCloud(width = 512, height=512).generate(spam_words)
plt.figure(figsize=(10,8), facecolor='k')
plt.imshow(spamwc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

TypeError: sequence item 0: expected str instance, ResultSet found

In [ ]:
-